# HW 1 - Building a network from scratch

* x1, x2, x3 = (1, 2, -1)
* Weights have been initated with 1. value
* Biases have been initated with 0 value
* The reuqired network must include 2 hidden layers, with 2 neurons in each layer
* One dimension output layer
* true y value = 1


# Loading required packages 

In [ ]:
import pandas as pd
import numpy as np
import numpy.typing as npt

# Utils functions

In [163]:
def ReLU(x):
    return np.clip(x, 0, None)

def d_ReLU(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def loss(y_pred: npt.ArrayLike, y_true: npt.ArrayLike):
    return ((y_pred - y_true) ** 2).mean()

def d_loss(y_pred: npt.ArrayLike, y_true: npt.ArrayLike):
    n = y_true.shape[0]
    gradient = 2. * (y_pred - y_true) / n
    return gradient

# The *Layer* class - OOP

 * The learning rate is set to 0.01
 * The activation function is ReLU

In [175]:
class Layer:

    activationFunctions = {
        'ReLU': (ReLU, d_ReLU)
    }
    
    lr = 0.1
    
    def __init__(self, neurons: int, inputs: int, activation: str = 'ReLU'):
        """
        The layer instance should be initiated with the input count and number of neurons.
        """
        self.W = np.ones((neurons, inputs))
        self.b = np.zeros((neurons, 1))
        self.act, self.d_act = self.activationFunctions.get(activation)
        
    def forward(self, A_prev: npt.ArrayLike) -> npt.ArrayLike:
        """
        Each forward step will calculate the next layer neurin as a function of its prev input and current 
        weight dot product, on top of that, the activation function will be applied.
        """
        self.A_prev = A_prev
        self.Z = np.dot(self.W, self.A_prev) + self.b
        self.A = self.act(self.Z)
        return self.A
        
    def backprop(self, dA: npt.ArrayLike) -> npt.ArrayLike:
        dZ = np.multiply(self.d_act(self.Z), dA)
        dW = 1/dZ.shape[1] * np.dot(dZ, self.A_prev.T)
        db = 1/dZ.shape[1] * np.sum(dZ, axis=1, keepdims=True)
        dA_prev = np.dot(self.W.T, dZ)
        
        self.W = self.W - self.lr * dW
        self.b = self.b - self.lr * db
        
        return dA_prev

In [223]:
def logit(index, text, value = None, add_sep: bool = False):
    print(f"{text} {index}: \n {value if value is not None else ''}")
    if add_sep:
        print(f"{'*'*50}")

# Given inputs

x_train = np.array([[1.0], [2.0], [-1.0]])
y_train = np.array([[0]]) 

m = 1
epochs = 1

layers = [Layer(2, 3, 'ReLU'), Layer(2, 2, 'ReLU'), Layer(1, 2, 'ReLU')]
costs = []

for i, epoch in enumerate(range(epochs)):
    logit(i, "Epoch", add_sep=True)
    # FORWARD
    A = x_train
    for j, layer in enumerate(layers):
        A = layer.forward(A)
        logit(j, "Layer", A)
    
    # COSTS
    cost = loss(y_train, A)
    logit(i, "Epoch Cost", cost)
    costs.append(cost)
    
    # BACKPROP
    dA = d_loss(y_train, A)
    for k, layer in enumerate(reversed(layers)):
        dA = layer.backprop(dA)
        logit(k, "Backprop")

print(f"Final param values:\n")
print(f"Weight: {layer.W}")
print(f"Biases: {layer.b}")

Epoch 0: 
 
**************************************************
Layer 0: 
 [[2.]
 [2.]]
Layer 1: 
 [[4.]
 [4.]]
Layer 2: 
 [[8.]]
Epoch Cost 0: 
 64.0
Backprop 0: 
 
Backprop 1: 
 
Backprop 2: 
 
Final reaults

Weight: [[ 4.2  7.4 -2.2]
 [ 4.2  7.4 -2.2]]
Biases: [[3.2]
 [3.2]]
